In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("train_data.csv")
data.drop(["rRange","index"],axis=1,inplace=True)
data.head()

,Type,rValue,centerDistance
0,0,0,59.0
1,1,0,63.0
2,2,0,298.0
3,0,0,207.0
4,3,4,37.0


In [4]:
len(set(data.rValue.tolist())),len(data)

(63, 231573)

In [5]:
X = data[["Type","centerDistance"]]
Y = data.rValue.tolist()

In [6]:
from sklearn.model_selection import train_test_split
# 区分训练集合 测试集
x_train, x_test, y_train, y_test = train_test_split(X, Y) 

In [12]:
x_train

,Type,centerDistance
59118,0,129.0
86584,18,13.0
197954,92,253.0
54472,1,60.0
118773,0,74.0
...,...,...
71417,79,43.0
196096,84,331.0
165148,0,132.0
209879,74,45.0


In [7]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder


# encode class values as integers
# y值 热编码 ；就是 多分类 将 y值 用0，1表示 ，比如 3分类 0类别就是[1,0,0],1类别就是[0,1,0],2类别就是[0,0,1]
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y)
# convert integers to dummy variables (one hot encoding) 得到的 dummy_y 就是热编码的
dummy_y = np_utils.to_categorical(encoded_Y)

# define model structure
def baseline_model():
    # 构造模型
    model = Sequential()
    # 添加层 ，层的 输入是 2维，输出是20维，使用 relu激活函数
    model.add(Dense(output_dim=20, input_dim=2, activation='relu'))
    # 添加 Dropout 层，随机 将20%的值 赋值为0，防止过拟合
    model.add(Dropout(0.2))
    # 全连接层 用来预测 ，输入是上一层的 输出20维，输出是 类别个数 ，多分类使用softmax做激活函数
    model.add(Dense(output_dim=len(set(data.rValue.tolist())), input_dim=20, activation='softmax'))
    # Compile model 多分类使用 categorical_crossentropy做损失函数，优化器使用adam，指标使用accuracy即准确率
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
#
# estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=40, batch_size=256)
# splitting data into training set and test set. If random_state is set to an integer, the split datasets are fixed.
# 切分数据 训练：验证 = 7：3 ，random_state是种子，可以使每次训练结果一致
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.3, random_state=0)

# 构造网络
model = baseline_model()

# 训连模型 batch_size 是每次用多少样本，epochs 是训练的轮数，validation_data 是验证数据集
history = model.fit(X_train, Y_train, batch_size=256, epochs=10,
            verbose=1, validation_data=(X_test, Y_test))


Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

Train on 162101 samples, validate on 69472 samples
Epoch 1/10
162101/162101 [==============================] - 1s 8us/step - loss: 9.0351 - accuracy: 0.1883 - val_loss: 2.8232 - val_accuracy: 0.3172
Epoch 2/10
162101/162101 [==============================] - 1s 7us/step - loss: 2.8245 - accuracy: 0.3051 - val_loss: 2.6909 - val_accuracy: 0.3173
Epoch 3/10
162101/162101 [==============================] - 1s 9us/step - loss: 2.6669 - accuracy: 0.3173 - val_loss: 2.6149 - val_accuracy: 0.3173
Epoch 4/10
162101/162101 [==============================] - 1s 8us/step - loss: 2.6299 - accuracy: 0.3175 - val_loss: 2.6008 - val_accuracy: 0.3173
Epoch 5/10
162101/162101 [==============================] - 1s 7us/step - loss: 2.6215 - accuracy: 0.3175 - val_loss: 2.5947 - val_accuracy: 0.3173
Epoch 6/10
162101/162101 [==============================] - 1s 8us/step - loss: 2.6183 - accuracy: 0.3175 - val_loss: 2.5952 - val_accuracy: 0.3173
Epoch 7/10
162101/162101 [==============================] - 1

In [8]:
score = model.evaluate(X_test, Y_test, batch_size=256)
print('test loss:', score[0])
print('test accuracy:', score[1])

69472/69472 [==============================] - 0s 3us/step
test loss: 2.5854503442921546
test accuracy: 0.3172501027584076


In [35]:
import numpy as np
encoder.inverse_transform(model.predict_classes(np.array([[24.,41.]])))

array([0])